In [2]:
import numpy as np
import random
from spacy.tokenizer import Tokenizer
from spacy.lang.pl import Polish
import copy
from math import floor

## ZAD1

In [3]:
def delta1(a, b):
    if a == b:
        return 0
    else:
        return 1


def edit_distance(x, y, delta):
    edit_table = [[None for i in range(len(y)+1)] for j in range(len(x)+1)]
    edit_table[0][0] = ((0, 0), 0)
    for i in range(1, len(edit_table)):
        edit_table[i][0] = ((0, i - 1), i)
    for j in range(1, len(edit_table[0])):
        edit_table[0][j] = ((j - 1, 0), j)

    for i in range(1, len(edit_table)):
        for j in range(1, len(edit_table[0])):
            if edit_table[i-1][j][1] < edit_table[i-1][j-1][1] and edit_table[i-1][j][1] < edit_table[i][j-1][1]:  #up/del
                edit_table[i][j] = ((i-1, j), edit_table[i-1][j][1] + 1)
            elif edit_table[i][j-1][1] < edit_table[i-1][j-1][1] and edit_table[i][j-1][1] < edit_table[i-1][j][1]:  #left/ins
                edit_table[i][j] = ((i, j-1), edit_table[i][j-1][1] + 1)
            else:  #diag/sub
                edit_table[i][j] = ((i - 1, j - 1), edit_table[i - 1][j - 1][1] + delta(x[i-1], y[j-1]))

    path = []
    i = len(edit_table) - 1
    j = len(edit_table[0]) - 1
    while i != 0 or j != 0:
        if i - edit_table[i][j][0][0] == 1:
            if j - edit_table[i][j][0][1] == 1:
                if x[i-1] == y[j-1]:
                    path.append((edit_table[i][j][0], 0, "do nothing"))
                else:
                    path.append((edit_table[i][j][0], 1, "replace " + x[i-1] + " with " + y[j-1]))
            else:
                path.append((edit_table[i][j][0], 2, "delete " + x[i-1]))
        else:
            path.append((edit_table[i][j][0], 3, "add " + y[j-1]))
        i, j = edit_table[i][j][0][0], edit_table[i][j][0][1]

    path.reverse()

    return path, edit_table[-1][-1][1]



In [4]:
_, distance = edit_distance("los", "kloc", lambda a, b: 0 if a == b else 1)
print(distance)

2


## ZAD2

In [13]:
def correction(x):
    if x == 3:
        return -1
    else:
        return 0


def print_path(x, y, path):
    pos = 0
    change = list(x)
    print("".join(change))
    for step in path:
        i, j = step[0][0], step[0][1]
        if step[1] == 0: #do nothing
            continue
        elif step[1] == 1:  #replace
            change[i + pos] = y[j]
        elif step[1] == 2:  #delete
            change.pop(i + pos)
            pos -= 1
        elif step[1] == 3:  #add
            change.insert(i + pos, y[j])
            pos += 1
        if step[1] != 2:
            print("".join(change[:i + pos + correction(step[1])]) + "*" + change[i + pos + correction(step[1])] + "*" + "".join(change[i + 1 + pos + correction(step[1]):]) + " " + step[2])
        else:
            print("".join(change[:i + pos + correction(step[1])]) + "*" + "".join(change[i + 1 + pos + correction(step[1]):]) + " " + step[2])
    print(y)

In [14]:
path, distance = edit_distance("kloc", "los", lambda a, b: 0 if a == b else 1)
print_path("kloc", "los", path)

kloc
lo*loc delete k
lo*s* replace c with s
los


## ZAD 3

In [15]:
pairs = [("los", "kloc"), ("Łódź", "Lodz"), ("kwintesencja", "quintessence"), ("ATGAATCTTACCGCCTCG", "ATGAGGCTCTGGCCCCTG")]
for string1, string2 in pairs:
    path, distance = edit_distance(string1, string2, lambda a, b: 0 if a == b else 1)
    print_path(string1, string2, path)
    print()

los
*k*los add k
klo*c* replace s with c
kloc

Łódź
*L*ódź replace Ł with L
L*o*dź replace ó with o
Lod*z* replace ź with z
Lodz

kwintesencja
*q*wintesencja replace k with q
q*u*intesencja replace w with u
quintes*s*encja add s
quintessen*a delete j
quintessenc*e* replace a with e
quintessence

ATGAATCTTACCGCCTCG
ATGA*G*TCTTACCGCCTCG replace A with G
ATGAG*G*CTTACCGCCTCG replace T with G
ATGAGGCT*C*TACCGCCTCG add C
ATGAGGCTCT*G*CCGCCTCG replace A with G
ATGAGGCTCTG*G*CGCCTCG replace C with G
ATGAGGCTCTGGC*C*CCTCG replace G with C
ATGAGGCTCTGGCCCC*G delete C
ATGAGGCTCTGGCCCCTG



## ZAD 4

In [28]:
def lcs(x, y):
    lcs_table = np.empty((len(x) + 1, len(y) + 1), dtype=int)
    for i in range(0, len(lcs_table)):
        lcs_table[i][0] = 0
    for j in range(0, len(lcs_table[0])):
        lcs_table[0][j] = 0

    for i in range(1, len(lcs_table)):
        for j in range(1, len(lcs_table[0])):
            if x[i - 1] == y[j - 1]:
                lcs_table[i][j] = lcs_table[i - 1][j - 1] + 1
            else:
                lcs_table[i][j] = max(lcs_table[i - 1][j],
                                      lcs_table[i][j - 1])
    subsequence = []
    i = len(lcs_table) - 1
    j = len(lcs_table[0]) - 1
    while i != 0 or j != 0:
        # Moving to the same value
        if i > 0:
            if lcs_table[i - 1][j] == lcs_table[i][j]:  # up
                i -= 1
                continue
        if j > 0:
            if lcs_table[i][j - 1] == lcs_table[i][j]:  # left
                j -= 1
                continue
            # Moving to lower value
            subsequence.append(x[i - 1])
            i -= 1
            j -= 1

    subsequence.reverse()
    return "".join(subsequence), lcs_table[-1][-1]

def lcs_token(x, y):
    lcs_table = np.empty((len(x) + 1, len(y) + 1))
    for i in range(0, len(lcs_table)):
        lcs_table[i][0] = 0
    for j in range(0, len(lcs_table[0])):
        lcs_table[0][j] = 0

    for i in range(1, len(lcs_table)):
        for j in range(1, len(lcs_table[0])):
            if x[i - 1] == y[j - 1]:
                lcs_table[i][j] = lcs_table[i - 1][j - 1] + 1
            else:
                lcs_table[i][j] = max(lcs_table[i - 1][j],
                                      lcs_table[i][j - 1])

    return lcs_table[-1][-1]

In [12]:
subsequence, length = lcs("los", "kloc")
print(f"\"{subsequence}\", {length}")

"lo", 2


## ZAD 5

In [23]:
with open("romeo-i-julia-700.txt", "r", encoding="UTF-8") as file:
    string = file.read()
nlp = Polish()
tokenizer = Tokenizer(nlp.vocab)
tokens = tokenizer(string)


## ZAD 6

In [25]:
tokens1, tokens2 = list(tokens), list(tokens)

for _ in range(floor(len(tokens) * 0.03)):
    tokens1.remove(random.choice(tokens1))
    tokens2.remove(random.choice(tokens2))

## ZAD 7

In [30]:
lcs_token(tokens1, tokens2)

2139.0

## ZAD 8

In [32]:
def diff(x, y):
    lcs_table = np.empty((len(x) + 1, len(y) + 1))
    for i in range(0, len(lcs_table)):
        lcs_table[i][0] = 0
    for j in range(0, len(lcs_table[0])):
        lcs_table[0][j] = 0

    for i in range(1, len(lcs_table)):
        for j in range(1, len(lcs_table[0])):
            if x[i - 1] == y[j - 1]:
                lcs_table[i][j] = lcs_table[i - 1][j - 1] + 1
            else:
                lcs_table[i][j] = max(lcs_table[i - 1][j],
                                      lcs_table[i][j - 1])
    subsequence = []
    i = len(lcs_table) - 1
    j = len(lcs_table[0]) - 1
    while i != 0 or j != 0:
        # Moving to the same value
        if i > 0:
            if lcs_table[i - 1][j] == lcs_table[i][j]:  # up
                i -= 1
                continue
        if j > 0:
            if lcs_table[i][j - 1] == lcs_table[i][j]:  # left
                j -= 1
                continue
            # Moving to lower value
            subsequence.append(x[i - 1])
            i -= 1
            j -= 1

    subsequence.reverse()

    i = 0
    j = 0
    for tok in subsequence:
        while x[i] != tok:
            print("< " + str(i) + ": " + x[i])
            i += 1
        i += 1
        while y[j] != tok:
            print("> " + str(j) + ": " + y[j])
            j += 1
        j += 1
    while i < len(x):
        print("< " + str(i) + ": " + x[i])
        i += 1
    while j < len(y):
        print("> " + str(j) + ": " + y[j])
        j += 1

In [33]:
token_lines1, token_lines2 = [], []
line = ""
for token in tokens1:
    if str(token)[0] == "\n":  #end of line
        token_lines1.append(line)
        line = ""
    else:
        line += "".join(token.text_with_ws)

for token in tokens2:
    if str(token)[0] == "\n":  #end of line
        token_lines2.append(line)
        line = ""
    else:
        line += "".join(token.text_with_ws)

diff(token_lines1, token_lines2)

< 1: Romeo i Julia
< 2: tłum. Józef Paszkowski
> 1: Romeo Juliatłum. Józef Paszkowski
< 6: * — młody Weroneńczyk szlachetnego rodu, krewny księcia
> 5: * PARYS — młody Weroneńczyk szlachetnego rodu, krewny księcia
< 9: * ROMEO — syn Montekiego* MERKUCJO krewny księcia
> 8: * ROMEO — syn Montekiego
> 9: * MERKUCJO — krewny księcia
< 12: * LAURENTY — ojciec franciszkanin* JAN — brat z tegoż zgromadzenia
> 12: * LAURENTY — ojciec franciszkanin
> 13: * JAN — brat z tegoż zgromadzenia
< 15: * ABRAHAM — służący 
> 16: * ABRAHAM — służący Montekiego
< 20: * DOWÓDCA WARTY
> 21: * DOWÓDCA 
< 25: Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół obu domów, maski, straż wojskowa i inne osoby.
< 26: Rzecz odbywa się przez większą sztuki w Weronie, przez część piątego aktu w Mantui.
< 27: PROLOG
> 26: * Obywatele weroneńscy, różne osoby płci obojej, liczący się do przyjaciół domów, maski, straż wojskowa i inne osoby.
> 27: Rzecz odbywa się przez część sztuki w Weronie, przez 